In [ ]:
# install required libraries
!pip install PyPDF2
!pip install langchain
!pip install pypdf
!pip install sentence-transformers
!pip install faiss-cpu

In [2]:
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

import os

from langchain.document_loaders import Docx2txtLoader
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader

from langchain import HuggingFaceHub
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains.question_answering import load_qa_chain
import os
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "YOUR_API_TOKEN"

# Retrieval-Augmented Generation(RAG) for Large Language Models

[Paper link](https://arxiv.org/abs/2312.10997)

<img src="../images/rag_hyde_hugginface/rag_archi.jpeg" width="700">

Taking open source models from hugging face [link](https://huggingface.co/models)

In [3]:
llm_small=HuggingFaceHub(
    repo_id="google/flan-t5-small",
    model_kwargs={"temperature":0.2, "max_length":512}
    )
llm_qa=HuggingFaceHub(
    repo_id="ashishkat/questionAnswer",
    model_kwargs={"temperature":0.2, "max_length":512}
    )
llm_xxl=HuggingFaceHub(
    repo_id="google/flan-t5-xxl",
    model_kwargs={"temperature":0.2, "max_length":512}
    )
llm_mistral=HuggingFaceHub(
    repo_id="mistralai/Mistral-7B-Instruct-v0.1",
    model_kwargs={"temperature":0.1, "max_length":512}
    )

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.huggingface_hub.HuggingFaceHub` was deprecated in langchain-community 0.0.21 and will be removed in 0.2.0. Use HuggingFaceEndpoint instead.
  warn_deprecated(


In [4]:
query = "What is performance of model on detecting brick kilns"
chain = load_qa_chain(llm_mistral, chain_type="stuff")
base_response = chain.run(input_documents=[], question=query)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


In [5]:
base_response

"Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\n\n\nQuestion: What is performance of modele on detecting brick kilns\nHelpful Answer: The performance of the model on detecting brick kilns can be evaluated by calculating the accuracy, precision, recall, and F1 score. These metrics can be calculated by comparing the predicted results of the model with the actual results. The accuracy measures the percentage of correct predictions, while the precision measures the percentage of true positive predictions among all positive predictions. The recall measures the percentage of true positive predictions among all actual positive cases. The F1 score is the harmonic mean of precision and recall,"

In [6]:
split_text = base_response.split("Question:")
question_part = split_text[1].split("Helpful Answer:")

question = question_part[0].strip()
base_answer = question_part[1].strip()

print("Question:", question)
print("Helpful Answer:", base_answer)

Question: What is performance of modele on detecting brick kilns
Helpful Answer: The performance of the model on detecting brick kilns can be evaluated by calculating the accuracy, precision, recall, and F1 score. These metrics can be calculated by comparing the predicted results of the model with the actual results. The accuracy measures the percentage of correct predictions, while the precision measures the percentage of true positive predictions among all positive predictions. The recall measures the percentage of true positive predictions among all actual positive cases. The F1 score is the harmonic mean of precision and recall,


In [7]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# # Path to the database 'docs' directory in your Google Drive
# docs_path = '/content/drive/MyDrive/Colab Notebooks/langchain/docs/'

In [8]:
# documents = []
# for file in os.listdir(docs_path):
#     if file.endswith('.pdf'):
#         pdf_path = docs_path + file
#         loader = PyPDFLoader(pdf_path)
#         documents.extend(loader.load())
#     elif file.endswith('.docx') or file.endswith('.doc'):
#         doc_path = docs_path + file
#         loader = Docx2txtLoader(doc_path)
#         documents.extend(loader.load())
#     elif file.endswith('.txt'):
#         text_path = docs_path + file
#         loader = TextLoader(text_path)
#         documents.extend(loader.load())

# len(documents)

Doing for single pdf

In [9]:
documents = []
loader = PyPDFLoader('/content/drive/MyDrive/Colab Notebooks/langchain/docs/NeurIPS23_Workshop_Accepted_BrickKilns.pdf')
documents.extend(loader.load())
len(documents)

11

In [10]:
documents[0]

Document(page_content='NeurIPS 2023 Workshop on Adaptive Experimental Design and Active Learning in the Real World\nTowards Scalable Identification of Brick Kilns from Satellite\nImagery with Active Learning\nAditi Agarwal, Suraj Jaiswal, Madhav Kanda, Dhruv Patel, Rishabh Mondal,\nVannsh Jani, Zeel B Patel, Nipun Batra\nIndian Institute of Technology, Gandhinagar\nSarath Guttikunda\nUrban EmissionsAbstract\nAir pollution is a leading cause of death globally, especially in south-east Asia. Brick 1\nproduction contributes significantly to air pollution. However, unlike other sources such 2\nas power plants, brick production is unregulated and thus hard to monitor. Traditional 3\nsurvey-based methods for kiln identification are time and resource-intensive. Similarly, it 4\nis time-consuming for air quality experts to annotate satellite imagery manually. Recently, 5\ncomputer vision machine learning models have helped reduce labeling costs, but they need 6\nsufficiently large labeled imag

In [11]:
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=1000,
    chunk_overlap=200,
    length_function = len,

)
chunked_documents = text_splitter.split_documents(documents)
len(chunked_documents)

37

Loading the embeddings from the model

In [12]:
embeddings = HuggingFaceEmbeddings()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [13]:
# directly load if you have saved db
db_hf = FAISS.from_documents(chunked_documents, embeddings)

In [14]:
# # https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.faiss.FAISS.html#langchain.vectorstores.faiss.FAISS.from_documents
# db_hf.save_local('/content/drive/MyDrive/Colab Notebooks/langchain/faiss_index_docstore_mapping_hf')  # Save FAISS index, docstore, and index_to_docstore_id to disk.

In [11]:
embeddings = HuggingFaceEmbeddings()
db_hf = FAISS.load_local('/content/drive/MyDrive/Colab Notebooks/langchain/faiss_index_docstore_mapping_hf', embeddings)

In [15]:
top_docs = db_hf.similarity_search(query, k=5)
for i in top_docs:
  print(i,"\n\n")

page_content='1000 images, and after manual inspection, we found 996 of them to be correctly classified. 281\nThus through our approach we were able to reduce the annotation 282\nBrick Kilns\nIndia\nFigure A: We initially manually located approximately 189 brick kilns in the Indo-Gangetic\nplain. Subsequently, our model automatically detected an additional 704 new brick kilns in\nthe vicinity of the manually identified ones, as illustrated in the figure\nA.4 Deployment 283\nWe deploy a web application on Streamlit, as depicted in Figure 1, offering users an acces- 284\nsible and interactive interface for brick kiln detection in a given area of interest. Once the 285\nbounding box is defined, our model identifies brick kilns within this area and provides the 286\ncoordinates of the brick kilns. Grad-CAM (Selvaraju et al., 2019) visuals accompany these 287\non the original brick kiln image to highlight the areas where the model focuses. 288\n11' metadata={'source': '/content/drive/MyDriv

In [16]:
chain = load_qa_chain(llm_mistral, chain_type="stuff")
response = chain.run(input_documents=top_docs, question=query)

In [17]:
response

"Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\n1000 images, and after manual inspection, we found 996 of them to be correctly classified. 281\nThus through our approach we were able to reduce the annotation 282\nBrick Kilns\nIndia\nFigure A: We initially manually located approximately 189 brick kilns in the Indo-Gangetic\nplain. Subsequently, our model automatically detected an additional 704 new brick kilns in\nthe vicinity of the manually identified ones, as illustrated in the figure\nA.4 Deployment 283\nWe deploy a web application on Streamlit, as depicted in Figure 1, offering users an acces- 284\nsible and interactive interface for brick kiln detection in a given area of interest. Once the 285\nbounding box is defined, our model identifies brick kilns within this area and provides the 286\ncoordinates of the brick kilns. Grad-CAM (Selvaraju et al., 2019) visuals 

In [18]:
split_text = response.split("Question:")
question_part = split_text[1].split("Helpful Answer:")

question = question_part[0].strip()
helpful_answer = question_part[1].strip()

print("Question:", question)
print("Helpful Answer:", helpful_answer)

Question: What is performance of modele on detecting brick kilns
Helpful Answer: The model was able to reduce the annotation of 281 images to 282.


## Hypothetical Document Embeddings~(HyDE) [Paper link](https://arxiv.org/abs/2212.10496)

<img src="../images/rag_hyde_hugginface/hyde_architecture.png" width="900">

In [19]:
query, base_answer

('What is performance of modele on detecting brick kilns',
 'The performance of the model on detecting brick kilns can be evaluated by calculating the accuracy, precision, recall, and F1 score. These metrics can be calculated by comparing the predicted results of the model with the actual results. The accuracy measures the percentage of correct predictions, while the precision measures the percentage of true positive predictions among all positive predictions. The recall measures the percentage of true positive predictions among all actual positive cases. The F1 score is the harmonic mean of precision and recall,')

In [20]:
top_docs = db_hf.similarity_search(query+base_answer, k=5)
for i in top_docs:
  print(i,"\n\n")

page_content='We show that using our methods, we need to annotate only a small number of images to 39\nobtain brick kiln locations in a new region. On performing active learning on the Indian 40\ndataset, we concluded that we needed 70% fewer samples than random to achieve a similar 41\nF1 score. We also find that we could reach 97% of optimal F1 score with active learning, 42\nwhereas random could reach only 90% with the same number of samples labeled. 43\nFinally, we have developed a web application2offering users an accessible and interactive 44\ninterface for brick kiln detection in a given region of interest. Figure 1 shows our web 45\napplication which takes in bounding boxes of the area of interest and detects the kilns 46\npresent in the region while also showing Grad-CAM (Selvaraju et al., 2019) visuals to 47\nhighlight the focus area of the model. Our work is fully reproducible, and we intend to 48\nrelease the scripts and data upon acceptance. 49\n2. Dataset 50' metadata={'s

In [21]:
chain = load_qa_chain(llm_mistral, chain_type="stuff")
response_hyde = chain.run(input_documents=top_docs, question=query)

In [22]:
response_hyde

"Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\nWe show that using our methods, we need to annotate only a small number of images to 39\nobtain brick kiln locations in a new region. On performing active learning on the Indian 40\ndataset, we concluded that we needed 70% fewer samples than random to achieve a similar 41\nF1 score. We also find that we could reach 97% of optimal F1 score with active learning, 42\nwhereas random could reach only 90% with the same number of samples labeled. 43\nFinally, we have developed a web application2offering users an accessible and interactive 44\ninterface for brick kiln detection in a given region of interest. Figure 1 shows our web 45\napplication which takes in bounding boxes of the area of interest and detects the kilns 46\npresent in the region while also showing Grad-CAM (Selvaraju et al., 2019) visuals to 47\nhighlight the fo

In [23]:
split_text = response_hyde.split("Question:")
question_part = split_text[1].split("Helpful Answer:")

question = question_part[0].strip()
helpful_answer = question_part[1].strip()

print("Question:", question)
print("Helpful Answer:", helpful_answer)

Question: What is performance of modele on detecting brick kilns
Helpful Answer: The model was able to reduce the annotation of brick kilns from 189 to 704 through active learning, achieving a similar F1 score with 70% fewer samples. The model was able to reach 97% of optimal F1 score with active learning, whereas random could reach only 90% with the same number of samples labeled. The model was also able to detect brick kilns in a new region with only a small number of annotated
